In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
df = pd.read_excel('travel_destination_dataset.xlsx')

In [3]:
def assign_tags(row, tag):
    return tag in row

In [4]:
# Load the data
data = df
# Preprocess the 'Tags' column
data['Tags'] = data['Tags'].apply(lambda x: ' '.join(x.split('|')))

# Encode categorical variables (Tags and State)
tfidf = TfidfVectorizer()
# tfidf_matrix = tfidf.fit_transform(data['Tags'] + ' ' + data['State'])
tfidf_matrix = tfidf.fit_transform(df["Tags"])
# Compute cosine similarity between TF-IDF vectors
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to recommend cities based on a specific city and number of cities
def recommend_city(city, num_cities, cosine_sim=cosine_sim):
    # Find the index of the provided city
    city_index = data[data['City'] == city].index[0]
    
    # Compute similarity scores between the provided city and all other cities
    sim_scores = cosine_sim[city_index]

    # Get indices of cities sorted by similarity score (excluding the provided city)
    sim_indices = sim_scores.argsort()[::-1]
    recommended_cities = []
    for idx in sim_indices:
        if idx != city_index:
            recommended_cities.append(data.iloc[idx]['City'])
        if len(recommended_cities) == num_cities:
            break
    return recommended_cities

# Example usage
city = 'Ganga Sagar (Sagardwip)'  # Example city provided by the user
num_cities = 12  # Example number of cities the user wants to see
recommended_cities = recommend_city(city, num_cities)
print("Recommended Cities for", city, ":", recommended_cities)


Recommended Cities for Ganga Sagar (Sagardwip) : ['Badrinath', 'Hajo', 'Shirdi', 'Rameshwaram', 'Vrindavan', 'Nashik', 'Mathura', 'Somnath', 'Prayagraj', 'Haridwar', 'Dwarka', 'Varanasi']


In [5]:
pickle.dump(cosine_sim,open('similarity_cities.pkl','wb'))